In [23]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# Read Excel data into DataFrames
df_species = pd.read_excel('species_present_at_mgp.xlsx')
df_color_coded = pd.read_excel('invasive plant priorities and visibility months static copy.xlsx')

# Remove spaces from column names
df_species.columns = df_species.columns.str.replace(' ', '')
df_color_coded.columns = df_color_coded.columns.str.replace(' ', '')

# Merge tables based on species column
merged_df = pd.merge(df_species, df_color_coded, on='Species', how='left')


# Load existing workbook
wb = load_workbook('invasive plant priorities and visibility months static copy.xlsx')
ws = wb.active  # Assuming you're working with the active sheet

# Create a summary table
summary_table = merged_df.copy()  # Copy the merged DataFrame

# Specify the columns to include in the summary table
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Iterate over months and retrieve color-coded data
for month in months:
    # Find the column letter for the month in the original workbook
    for col in ws.iter_cols(min_col=1, max_col=ws.max_column, min_row=1, max_row=1):
        for cell in col:
            if month.lower() in cell.value.lower():
                col_letter = cell.column_letter
                # Copy the cell values and colors to the summary table
                for i, species in enumerate(df_species['Species']):
                    if species in merged_df['Species'].values:
                        val = merged_df.loc[merged_df['Species'] == species, month].iloc[0]
                        summary_table.at[i, month + '_color'] = val
                        summary_table.at[i, month + '_color_fill'] = cell.fill.start_color.rgb  # Store the color RGB value
                    else:
                        summary_table.at[i, month + '_color'] = 'Default Value'  # Filling missing data with a default value
                        summary_table.at[i, month + '_color_fill'] = 'Default Color'  # Filling missing color with a default color
                break  # Move to the next month
        if month.lower() in cell.value.lower():
            break  # Exit the loop if the month column is found
# Save the modified workbook
wb.save('summary_table.xlsx')
